In [101]:
import pandas as pd
import numpy as np

데이터 세트 살펴보기

In [102]:
pop = pd.read_csv('state-population.csv')
areas = pd.read_csv('state-areas.csv')
abbrevs = pd.read_csv('state-abbrevs.csv')

print(pop.head())
print(areas.head())
print(abbrevs.head())


  state/region     ages  year  population
0           AL  under18  2012   1117489.0
1           AL    total  2012   4817528.0
2           AL  under18  2010   1130966.0
3           AL    total  2010   4785570.0
4           AL  under18  2011   1125763.0
        state  area (sq. mi)
0     Alabama          52423
1      Alaska         656425
2     Arizona         114006
3    Arkansas          53182
4  California         163707
        state abbreviation
0     Alabama           AL
1      Alaska           AK
2     Arizona           AZ
3    Arkansas           AR
4  California           CA


Population Dataframe에 전체 주의 이름을 제공하는 다대일 병합

In [103]:
merged = pd.merge(pop, abbrevs, how='outer', left_on='state/region', right_on='abbreviation')
merged = merged.drop('abbreviation', axis=1)
merged.head()

,state/region,ages,year,population,state
0,AK,total,1990,553290.0,Alaska
1,AK,under18,1990,177502.0,Alaska
2,AK,total,1992,588736.0,Alaska
3,AK,under18,1991,182180.0,Alaska
4,AK,under18,1992,184878.0,Alaska


null 값 가진 행 찾기

In [104]:
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

In [105]:
merged[merged['population'].isnull()].head()

,state/region,ages,year,population,state
1872,PR,under18,1990,NaN,NaN
1873,PR,total,1990,NaN,NaN
1874,PR,total,1991,NaN,NaN
1875,PR,under18,1991,NaN,NaN
1876,PR,total,1993,NaN,NaN


In [106]:
merged.loc[merged['state'].isnull(), 'state/region'].unique()

array(['PR', 'USA'], dtype=object)

푸에트리코가 주 이름 약어 키에 없는 문제 해결 - 적절한 항목 채워넣기

In [107]:
merged.loc[merged['state/region'] == 'PR' , 'state'] = 'Puerto Rico'
merged.loc[merged['state/region'] == 'USA' , 'state'] = 'United States'
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

면적 데이터가 포함된 결과를 병합

In [108]:
final = pd.merge(merged, areas, on='state', how='left')
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AK,total,1990,553290.0,Alaska,656425.0
1,AK,under18,1990,177502.0,Alaska,656425.0
2,AK,total,1992,588736.0,Alaska,656425.0
3,AK,under18,1991,182180.0,Alaska,656425.0
4,AK,under18,1992,184878.0,Alaska,656425.0


In [109]:
final.isnull().any()

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool

null 값 삭제

In [110]:
final['state'][final['area (sq. mi)'].isnull()].unique()

array(['United States'], dtype=object)

In [111]:
final.dropna(inplace=True)
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AK,total,1990,553290.0,Alaska,656425.0
1,AK,under18,1990,177502.0,Alaska,656425.0
2,AK,total,1992,588736.0,Alaska,656425.0
3,AK,under18,1991,182180.0,Alaska,656425.0
4,AK,under18,1992,184878.0,Alaska,656425.0


전체 인구 밀도 계산 후 내림차순 순위 배열(A)

In [112]:
data2010 = final.query("year == 2010 & ages == 'total'")
data2010.head()

,state/region,ages,year,population,state,area (sq. mi)
43,AK,total,2010,713868.0,Alaska,656425.0
51,AL,total,2010,4785570.0,Alabama,52423.0
141,AR,total,2010,2922280.0,Arkansas,53182.0
149,AZ,total,2010,6408790.0,Arizona,114006.0
197,CA,total,2010,37333601.0,California,163707.0


In [113]:
print(data2010.columns)

Index(['state/region', 'ages', 'year', 'population', 'state', 'area (sq. mi)'], dtype='object')


In [114]:
data2010.set_index('state', inplace=True)
density = data2010['population'] / data2010['area (sq. mi)']

In [115]:
density.sort_values(ascending=False, inplace=True)
density.head()

state
District of Columbia    8898.897059
Puerto Rico             1058.665149
New Jersey              1009.253268
Rhode Island             681.339159
Connecticut              645.600649
dtype: float64

In [116]:
density.tail()

state
South Dakota    10.583512
North Dakota     9.537565
Montana          6.736171
Wyoming          5.768079
Alaska           1.087509
dtype: float64

18세 미만 인구 밀도 계산 후 내림차순 순위 배열 (B)

In [117]:
data2010_18 = final.query("year == 2010 & ages == 'under18'")
data2010_18.head()

,state/region,ages,year,population,state,area (sq. mi)
42,AK,under18,2010,187902.0,Alaska,656425.0
50,AL,under18,2010,1130966.0,Alabama,52423.0
140,AR,under18,2010,711947.0,Arkansas,53182.0
148,AZ,under18,2010,1628563.0,Arizona,114006.0
196,CA,under18,2010,9284094.0,California,163707.0


In [118]:
data2010_18.set_index('state', inplace=True)
density_18 = data2010_18['population'] / data2010_18['area (sq. mi)']

In [119]:
density_18.sort_values(ascending=False, inplace=True)
density_18.head()

state
District of Columbia    1489.838235
Puerto Rico              255.176387
New Jersey               236.415157
Connecticut              146.859127
Rhode Island             144.393528
dtype: float64

B > A 인 주 5개 출력

In [127]:
print(data2010.head())
print(data2010_18.head())

           state/region   ages  year  population  area (sq. mi)
state                                                          
Alaska               AK  total  2010    713868.0       656425.0
Alabama              AL  total  2010   4785570.0        52423.0
Arkansas             AR  total  2010   2922280.0        53182.0
Arizona              AZ  total  2010   6408790.0       114006.0
California           CA  total  2010  37333601.0       163707.0
           state/region     ages  year  population  area (sq. mi)
state                                                            
Alaska               AK  under18  2010    187902.0       656425.0
Alabama              AL  under18  2010   1130966.0        52423.0
Arkansas             AR  under18  2010    711947.0        53182.0
Arizona              AZ  under18  2010   1628563.0       114006.0
California           CA  under18  2010   9284094.0       163707.0


In [128]:
print(density.head())
print(density_18.head())

state
District of Columbia    8898.897059
Puerto Rico             1058.665149
New Jersey              1009.253268
Rhode Island             681.339159
Connecticut              645.600649
dtype: float64
state
District of Columbia    1489.838235
Puerto Rico              255.176387
New Jersey               236.415157
Connecticut              146.859127
Rhode Island             144.393528
dtype: float64


In [130]:
print(density.index.equals(density_18.index))

False


In [136]:
density_sorted = density.sort_index()
density_18_sorted = density_18.sort_index()

print(density_sorted.head())
print(density_18_sorted.head())

state
Alabama        91.287603
Alaska          1.087509
Arizona        56.214497
Arkansas       54.948667
California    228.051342
dtype: float64
state
Alabama       21.573851
Alaska         0.286251
Arizona       14.284889
Arkansas      13.386992
California    56.711649
dtype: float64


B > A 인 주가 하나도 출력되지 않는 문제 발생

In [134]:
higher_density_states_18 = density_18_sorted[density_18_sorted > density_sorted]
print(higher_density_states_18.head())

Series([], dtype: float64)


In [124]:
sorted_higher_density_states_18 = higher_density_states_18.sort_values(ascending=False)

sorted_higher_density_states_18.head()

Series([], dtype: float64)

In [123]:
top5_states_18 = sorted_higher_density_states_18.head()
print(top5_states_18)

Series([], dtype: float64)


문제 해결 위해 밀도의 차이 그 자체에 집중하여 다시 접근

In [138]:
density_difference_18 = density_18_sorted - density_sorted
print(density_difference_18.head(10))

state
Alabama                  -69.713752
Alaska                    -0.801258
Arizona                  -41.929609
Arkansas                 -41.561675
California              -171.339692
Colorado                 -36.710634
Connecticut             -498.741522
Delaware                -355.288127
District of Columbia   -7409.058824
Florida                 -225.775145
dtype: float64


In [142]:
significant_difference_18 = density_difference_18.abs().sort_values(ascending=False).head(5)
print(significant_difference_18)

state
District of Columbia    7409.058824
Puerto Rico              803.488762
New Jersey               772.838111
Rhode Island             536.945631
Connecticut              498.741522
dtype: float64


 <h3> A vs B 문제의 결론</h3><br/>
- 18세 이하의 인구 밀도가 주 전체 인구 밀도 보다 높은 주는 미국에 존재하지 않음. <br/>
- 다만 18세 이하와 전체 연령의 인구 밀도가 큰 차이를 보이는 주는 차례로 <br/>

District of Columbia    7409.058824 <br/>
Puerto Rico              803.488762 <br/>
New Jersey               772.838111 <br/>
Rhode Island             536.945631 <br/>
Connecticut              498.741522 <br/>

를 상위 5개 주로 뽑을 수 있음. <br/>


2011, 2012년의 2010년 대비 전체 / 18세 미만 인구 밀도 순위 변동 계산

In [164]:
data_2010= final.query("year == 2010 & ages == 'total'" )
data_2010_18= final.query("year == 2010 & ages == 'under18'" )

data_2011= final.query("year == 2011 & ages == 'total'" )
data_2011_18= final.query("year == 2011 & ages == 'under18'" )

data_2012= final.query("year == 2012 & ages == 'total'" )
data_2012_18= final.query("year == 2012 & ages == 'under18'" )


In [165]:
data_2010.set_index('state', inplace=True)
data_2010_18.set_index('state', inplace=True)

density_2010 = data_2010['population'] / data_2010['area (sq. mi)']
density_2010_18 = data_2010_18['population'] / data_2010_18['area (sq. mi)']

data_2011.set_index('state', inplace=True)
data_2011_18.set_index('state', inplace=True)

density_2011 = data_2011['population'] / data_2011['area (sq. mi)']
density_2011_18 = data_2011_18['population'] / data_2011_18['area (sq. mi)']

data_2012.set_index('state', inplace=True)
data_2012_18.set_index('state', inplace=True)

density_2012 = data_2012['population'] / data_2012['area (sq. mi)']
density_2012_18 = data_2012_18['population'] / data_2012_18['area (sq. mi)']

In [166]:
density_2010.head()

state
Alaska          1.087509
Alabama        91.287603
Arkansas       54.948667
Arizona        56.214497
California    228.051342
dtype: float64

In [171]:
density_2010.sort_values(ascending=False, inplace=True)
density_2010_18.sort_values(ascending=False, inplace=True)

density_2011.sort_values(ascending=False, inplace=True)
density_2011_18.sort_values(ascending=False, inplace=True)

density_2012.sort_values(ascending=False, inplace=True)
density_2012_18.sort_values(ascending=False, inplace=True)


In [170]:
density_2010.head()

state
District of Columbia    8898.897059
Puerto Rico             1058.665149
New Jersey              1009.253268
Rhode Island             681.339159
Connecticut              645.600649
dtype: float64

In [172]:
density_2011.head()

state
District of Columbia    9112.117647
Puerto Rico             1048.813656
New Jersey              1013.143660
Rhode Island             679.838188
Connecticut              647.357143
dtype: float64

In [180]:
density_change_2011 = density_2011 - density_2010
density_change_2011_18 = density_2011_18 - density_2010_18

density_change_2012 = density_2012 - density_2010
density_change_2012_18 = density_2012_18 - density_2010_18


In [179]:
print("2010년 대비 2011년의 전체 연령 주별 인구 밀도 변화")
print(density_change_2011.sort_values(ascending=False).head(10))

2010년 대비 2011년의 전체 연령 주별 인구 밀도 변화
state
District of Columbia    213.220588
Maryland                  4.275651
Delaware                  4.234391
Massachusetts             4.075983
New Jersey                3.890392
Florida                   3.610633
California                2.046828
New York                  1.918311
Virginia                  1.904019
Connecticut               1.756494
dtype: float64


In [181]:
print("2010년 대비 2011년의 18세 이하 연령 주별 인구 밀도 변화")
print(density_change_2011_18.sort_values(ascending=False).head(10))

2010년 대비 2011년의 18세 이하 연령 주별 인구 밀도 변화
state
District of Columbia    38.191176
Texas                    0.196366
Hawaii                   0.144896
Utah                     0.098123
Oklahoma                 0.060527
Virginia                 0.059856
Washington               0.045903
Florida                  0.045896
North Carolina           0.036231
Colorado                 0.034188
dtype: float64


In [182]:
print("2010년 대비 2012년의 전체 연령 주별 인구 밀도 변화")
print(density_change_2012.sort_values(ascending=False).head(10))

2010년 대비 2012년의 전체 연령 주별 인구 밀도 변화
state
District of Columbia    416.205882
Delaware                  8.875128
Maryland                  7.872572
Massachusetts             7.772620
New Jersey                7.457235
Florida                   7.218817
California                4.069936
Virginia                  3.792724
North Carolina            3.508500
Georgia                   3.405023
dtype: float64


In [184]:
print("2010년 대비 2012년의 18세 이하 연령 주별 인구 밀도 변화")
print(density_change_2012_18.sort_values(ascending=False).head(10))

2010년 대비 2012년의 18세 이하 연령 주별 인구 밀도 변화
state
District of Columbia    93.132353
Texas                    0.397590
Hawaii                   0.198408
Florida                  0.196007
Utah                     0.183254
Virginia                 0.147256
Oklahoma                 0.120567
Washington               0.098383
North Dakota             0.093149
Colorado                 0.059990
dtype: float64


<h3>2010년 대비 인구 밀도 변화 문제 결론 - 연도별 변동 폭 상위 5개주</h3> <br/>
- 2011년 전체 연령 기준 : District of Columbia, Maryland, Delaware, Massachusetts, New Jersey, Florida <br/>
- 2011년 18세 이하 연령 기준 : District of Columbia, Texas, Hawaii, Utah, Oklahoma <br/> <br/>
- 2012년 전체 연령 기준 : District of Columbia, Delaware, Maryland, Massachusetts, New Jersey <br/>
- 2012년 18세 이하 연령 기준 : District of Columbia, Texas, Hawaii,Florida, Utah